In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torchvision import transforms
from utils.getter import *

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def get_score(img):
    url = 'http://84c845071c2c.ngrok.io/'
    my_img = {'image': open(img, 'rb')}
    r = requests.post(url, files=my_img)
    return float(r.json()['score'])

In [5]:
print('Old score: ', get_score('datasets/koni/pp2020_test/Places365_val_00019377.png'))
print('New score: ',get_score('test/output_image2.jpg'))

Old score:  67.33811
New score:  66.8563


In [4]:
from models.BIQA_model.inceptionresnetv2 import inceptionresnetv2
class model_qa(nn.Module):
    def __init__(self,num_classes,**kwargs):
        super(model_qa,self).__init__()
        base_model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')
        self.base= nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(1536, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),         
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes),
        )
    
    def forward(self,x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    
    def freeze(self):
        for param in self.net.parameters():
            param.requires_grad = False

In [5]:
class BIQA(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = model_qa(num_classes = 1)
        self.net.load_state_dict(torch.load('./BIQA_model/KonCept512.pth'))
        self.net.freeze()
    
    #def zero_grad(self):
        #pass
    
    def forward(self, x):
        return self.net(x)

In [6]:
#Define the Convolutional Autoencoder
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)

        self.name = 'cae'
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = torch.sigmoid(self.t_conv2(x))
              
        return x

In [7]:
class conv_block_nested(nn.Module):
    
    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output
    
#Nested Unet

class NestedUNet(nn.Module):
    """
    Implementation of this paper:
    https://arxiv.org/pdf/1807.10165.pdf
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(NestedUNet, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)


    def forward(self, x):
        
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)
        return output

In [8]:
class ImageFolder(data.Dataset):
    def __init__(self, path, transforms = None):
        self.path = path
        self.transforms = transforms
        self.load_data()
    
    def load_data(self):
        self.fns = [os.path.join(self.path, i) for i in os.listdir(self.path)]
        
    def __getitem__(self, idx):
        item = self.fns[idx]
        img = Image.open(item).convert('RGB')
        
        if self.transforms is not None:
            img = self.transforms(img)['img']

        label = img.detach().clone()
        return img, label
    
    def __len__(self):
        return len(self.fns)

    def collate_fn(self, batch):
        imgs = torch.stack([i[0] for i in batch])
        labels = torch.stack([i[1] for i in batch])
        return {'imgs': imgs, 'labels': labels}

In [9]:
def denormalize(img, mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]): 
    mean = np.array(mean)
    std = np.array(std)
    img_show = img.clone()
    if img_show.shape[0] == 1:
        img_show = img_show.squeeze(0)
    img_show = img_show.numpy().transpose((1,2,0))
    img_show = (img_show * std+mean)
    img_show = np.clip(img_show,0,1)
    return img_show

In [10]:
data_transforms = Compose([
        #transforms.CenterCrop(),
        Resize((384, 512)),
        ToTensor(),
        Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

In [11]:
trainset = ImageFolder(path = 'datasets/koni/pp2020_dev', transforms = data_transforms)
valset = ImageFolder(path = 'datasets/koni/pp2020_test', transforms = data_transforms)
trainloader = data.DataLoader(trainset, batch_size=4, collate_fn = trainset.collate_fn)
valloader = data.DataLoader(trainset, batch_size=4, collate_fn = valset.collate_fn)

In [12]:
def train_epoch(epoch, NUM_EPOCHS, model, trainloader, optimizerr, criterion):
    model.train()
    print_per_iter = 10
    running_loss = {
        'T': 0
        }
    running_time = 0
    for idx, batch in enumerate(trainloader):
        optimizer.zero_grad()
        inputs = batch['imgs'].to(device)
        targets = batch['labels'].to(device)

        start_time = time.time()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        end_time = time.time()

        running_loss['T'] += loss.item()
        running_time += end_time-start_time

        if idx % print_per_iter == 0:
            for key in running_loss.keys():
                running_loss[key] /= print_per_iter

            loss_string = '{}'.format(running_loss)[1:-1].replace("'",'').replace(",",' ||')

            print('[{}/{}][{}/{}] || {} || Time: {}s'.format(epoch+1, NUM_EPOCHS, idx+1, len(trainloader), loss_string,  running_time))
            running_time = 0
            running_loss = {
                'T': 0
                }

In [13]:
def validate_epoch(epoch, NUM_EPOCHS, model, valloader, criterion):
    #Validate

    model.eval()
    running_loss = {
        'T': 0
        }
    running_time = 0
    print('=============================EVALUATION===================================')
    with torch.no_grad():
        start_time = time.time()
        for idx, batch in enumerate(valloader):

            inputs = batch['imgs'].to(device)
            targets = batch['labels'].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss['T'] += (loss.item())

            end_time = time.time()
            running_time += (end_time - start_time)

        for key in running_loss.keys():
            running_loss[key] /= len(valloader)
            loss_string = '{}'.format(running_loss)[1:-1].replace("'",'').replace(",",' ||')

        print('[{}/{}] || Validation || {} || Time: {}s'.format(epoch+1, NUM_EPOCHS, loss_string, running_time))
        print('==========================================================================')

        return running_loss['T']

In [14]:
def train(model, trainloader, valloader, optimizer, criterion, scheduler):
    global min_val_loss
    NUM_EPOCHS = 50
    
    print('===========================START TRAINING=================================') 
    for epoch in range(NUM_EPOCHS):
        train_epoch(epoch, NUM_EPOCHS,model, trainloader, optimizer, criterion)
        val_loss = validate_epoch(epoch, NUM_EPOCHS, model, valloader, criterion)
        scheduler.step() 
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            torch.save(model.state_dict(), 'weights/{}/{}_{}_{:10.4f}.pth'.format(model.name, model.name, epoch, min_val_loss))  

In [15]:
model = NestedUNet(3,3).to(device) #ConvAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 30, 0.1)

In [16]:
min_val_loss = 2.0
train(model, trainloader, valloader, optimizer, criterion, scheduler)

===========================START TRAINING=================================


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 3.00 GiB total capacity; 1.51 GiB already allocated; 306.80 MiB free; 1.53 GiB reserved in total by PyTorch)

In [ ]:
img_ = Image.open('datasets/koni/pp2020_test/Places365_val_00015483.png')
img = data_transforms(img_)['img'].unsqueeze(0).to(device)

outputs = model(img)
outputs = outputs.detach().cpu()
outputs.shape

In [ ]:
img_show = denormalize(img.cpu())
img_show2 = denormalize(outputs)
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img_show)
plt.subplot(1,2,2)
plt.imshow(img_show2)
plt.show()